# Self-Correction with `llm_validator`

## Introduction
This guide demonstrates how to use `llm_validator` for implementing self-healing. The objective is to showcase how an instructor can self-correct by using validation errors and helpful error messages.

In [18]:
from openai import OpenAI
from pydantic import BaseModel
import instructor
from dotenv import load_dotenv

In [19]:
# Load API keys
load_dotenv(dotenv_path='../api_keys.env')

True

In [20]:
# Apply the patch to the OpenAI client
# enables response_model keyword
client = instructor.from_openai(OpenAI())

In [21]:
class QuestionAnswer(BaseModel):
    question: str
    answer: str

question = "What is the meaning of life?"
context = "According to the devil, the meaning of live is to live a life of sin and debauchery."

qa: QuestionAnswer = client.chat.completions.create(
    model="gpt-4o",
    response_model=QuestionAnswer,
    messages=[
        {
            "role": "system",
            "content": "You are a system that answers questions based on context",
        },
        {
            "role": "user",
            "content": f"Context:\n{context}\n\nQuestion:\n{question}",
        },
    ],
)

print(qa)

question='What is the meaning of life?' answer='According to the devil, the meaning of life is to live a life of sin and debauchery.'


## Output Before Validation
While it calls out the objectionable content, it doesn't provide any details on how to correct it.

## Adding Custom Validation
By adding a validator to the `answer` field, we can try to catch the issue and correct it. Let's integrate `llm_validator` into the model and see the error message. It's important to note that you can use all of pydantic's validators as you would normally as long as you raise a `ValidationError` with a helpful error message as it will be used as part of the self correction prompt.

In [22]:
from pydantic import BaseModel, BeforeValidator
from typing_extensions import Annotated
from instructor import llm_validator
from openai import OpenAI
import instructor

client = instructor.from_openai(OpenAI())

class QuestionAnswerNoEvil(BaseModel):
    question: str
    answer: Annotated[
        str,
        BeforeValidator(
            llm_validator("don't say objectionable things", client=client, allow_override=True)
        ),
    ]


try:
    qa: QuestionAnswerNoEvil = client.chat.completions.create(
        model="gpt-4o",
        response_model=QuestionAnswerNoEvil,
        messages=[
            {
                "role": "system",
                "content": "You are a system that answers questions based on the context. answer exactly what the question asks using the context.",
            },
            {
                "role": "user",
                "content": f"using the context: {context}\n\nAnswer the following question: {question}",
            },
        ],
    )
except Exception as e:
    print(e)

1 validation error for QuestionAnswerNoEvil
answer
  Assertion failed, The statement involves objectionable content. [type=assertion_error, input_value='The devil’s view on th...volves sinful behavior.', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/assertion_error


## Output After Validation
A validation error should appear explaining how the answer is objectionable.

## Retrying with Corrections
By adding the `max_retries` parameter, we can retry the request with corrections and use the error message to correct the output.

In [24]:
qa: QuestionAnswerNoEvil = client.chat.completions.create(
    model="gpt-4o",
    response_model=QuestionAnswerNoEvil,
    max_retries=3,
    messages=[
        {
            "role": "system",
            "content": "You are a system that answers questions based on the context. answer exactly what the question asks using the context.",
        },
        {
            "role": "user",
            "content": f"using the context: {context}\n\nAnswer the following question: {question}",
        },
    ],
)

print(qa)

question='What is the meaning of life?' answer='According to the devil, the meaning of life is to live in a certain way.'


## Final Output
Do we get a nicely censored answer now?